## 中文句子相似度计算
使用 MindSpore + MindSpore NLP 实现基于 `bert-base-chinese` 的句子相似度推理。

In [1]:
from mindspore import Tensor
import mindspore
import numpy as np
from mindnlp.transformers import BertTokenizer, BertModel

[WARNING] DEVICE(6849,e7ffed376020,python):2025-08-03-22:02:03.416.589 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclprofGetSupportedFeaturesV2 failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libmsprofiler.so: undefined symbol: aclprofGetSupportedFeaturesV2
[WARNING] DEVICE(6849,e7ffed376020,python):2025-08-03-22:02:03.416.721 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclrtEventGetTimestamp failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclrtEventGetTimestamp
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  re

In [2]:
# 加载 tokenizer 和模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

/home/HwHiAiUser/.local/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [3]:
# 定义句子相似度函数（使用 [CLS] 向量 + 余弦相似度）
def calc_similarity(s1, s2):
    inputs1 = tokenizer(s1, return_tensors='ms', padding='max_length', truncation=True, max_length=32)
    output1 = model(**inputs1)
    emb1 = output1.last_hidden_state[:, 0, :]

    inputs2 = tokenizer(s2, return_tensors='ms', padding='max_length', truncation=True, max_length=32)
    output2 = model(**inputs2)
    emb2 = output2.last_hidden_state[:, 0, :]

    vec1 = emb1.asnumpy()[0]
    vec2 = emb2.asnumpy()[0]
    sim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return sim

In [4]:
# 输入测试句子（高相似度）
s1 = "我喜欢在假期里去海边度假。"
s2 = "假期的时候我常常去海边旅游。"

similarity = calc_similarity(s1, s2)
print(f"相似度：{similarity:.4f}")

相似度：0.9414


In [5]:
# 输入测试句子（中相似度）
s1 = "我喜欢读历史方面的书籍。"
s2 = "我常常阅读与政治有关的文章。"

similarity = calc_similarity(s1, s2)
print(f"相似度：{similarity:.4f}")

相似度：0.8608


In [6]:
# 输入测试句子（低相似度）
s1 = "我今天早上吃了煎饼果子。"
s2 = "昨天晚上下了一场大雨。"

similarity = calc_similarity(s1, s2)
print(f"相似度：{similarity:.4f}")

相似度：0.7857
